In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

ModuleNotFoundError: No module named 'sklearn.model_selection'

In [3]:

path = '/Users/Venkat/Downloads'

url = ''
data=pd.read_csv(f'{path}/fetal_health.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   float64
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   float64
 8   mean_value_of_short_term_variability  

In [4]:
data

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [5]:
# basic pre-processing

data['fetal_health']=data['fetal_health'].astype(int)

In [6]:
data['fetal_health'].value_counts()

1    1655
2     295
3     176
Name: fetal_health, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)

names = np.array(data.columns)

for x in names:
    print(x)
    if x != 'fetal_health':
        data[x] = (data[x] - data[x].mean()) / (data[x].max() - data[x].min())

ModuleNotFoundError: No module named 'sklearn.preprocessing'

In [8]:
x=data.iloc[:,0:21].values
y=data.iloc[:,21].values

new_y = np.zeros((y.shape[0],3))
for i in range(y.shape[0]):
    idx = y[i]-1
    new_y[i][int(idx)] = 1
y = new_y

In [9]:
y.shape

(2126, 3)

In [10]:
#using stochastic gradient descent

import numpy as np
import pandas as pd

from scipy.special import softmax

from sklearn.datasets import load_iris

def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean() 

def sgd(
    gradient, x, y, start, learn_rate=0.1, batch_size=1, n_iter=50,
    tolerance=1e-06, dtype="float64", random_state=None
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]

    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    # Initializing the values of the variables
    vector = np.array(start, dtype=dtype_)

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Shuffle x and y
        rng.shuffle(xy)

        # Performing minibatch moves
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]

            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = -learn_rate * grad

            # Checking if the absolute difference is small enough
            if np.all(np.abs(diff) <= tolerance):
                break

            # Updating the values of the variables
            vector += diff

    return vector if vector.shape else vector.item()

ModuleNotFoundError: No module named 'sklearn.datasets'

In [11]:
#sigmoid activation function

def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def add_intercept(X):
    intercept = np.ones((X.shape[0], 1))
    return np.concatenate((intercept, X), axis=1)

In [11]:
class Multiclass:
    def __init__(self, idim=21, odim=3):
        self.W = 0.1*np.ones((idim+1, odim))
        print(self.W.shape)
        
    def fit(self, X, Y, lr = 0.1, epochs = 10):
        # self.W = sgd(ssr_gradient, X, Y, start=[0.5, 0.5])
        X = add_intercept(X)
        for epoch in range(epochs):
            out = self.forward(X)
            gradient = np.dot(X.T, (out - Y))/X.shape[0] # todo maybe mistake
            self.W -= gradient
            print((gradient**2).sum())

    def loss_plot(self):
        return self.loss_steps.plot(
            x='step', 
            y='loss',
            xlabel='step',
            ylabel='loss'
        )
    
    def forward(self, X):
        
        Z = sigmoid(X @ self.W) # todo
        return Z

    def predict(self, X):
        X = add_intercept(X)
        Z = self.forward(X)
        P = softmax(Z, axis=1)
        return np.argmax(P, axis=1)
    

        

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=41)

In [13]:
mlr=Multiclass()

mlr.fit(X_train,Y_train)
y_pred=mlr.predict(X_train)

(22, 3)
0.4243968940555736
0.24543545568883313
0.14806562644694077
0.09465588233856853
0.06412407644306838
0.04582370466516563
0.034354068178399914
0.026872113800833083
0.02181405619043241
0.018282681260326516


In [14]:
new_pred = np.zeros((y_pred.shape[0], 3))
for i in range(y_pred.shape[0]):
    idx = y_pred[i]
    new_pred[i][int(idx)] = 1
y_pred = new_pred

In [15]:
from sklearn.metrics import accuracy_score , f1_score
mlr_accuracy = accuracy_score(Y_train,y_pred)
mlr_f1 = f1_score(Y_train,y_pred, average='micro')
print('ACCURACY:',round(mlr_accuracy*100,2),'%')
print('F1 Score:',mlr_f1)

ACCURACY: 78.0 %
F1 Score: 0.78


In [16]:
#ReLu activation function

def relu(z):
    return z*(z>0)

class MulticlassReLu:
    def __init__(self, idim=21, odim=3):
        self.W = 0.01*np.ones((idim+1, odim))
        print(self.W.shape)
        
    def fit(self, X, Y, lr = 0.01, epochs = 10):
        # self.W = sgd(ssr_gradient, X, Y, start=[0.5, 0.5])
        X = add_intercept(X)
        for epoch in range(epochs):
            out =softmax(self.forward(X), axis=1) 
            gradient = (X.T @ ((out - Y)*(out > 0))).mean(axis=0)  # todo check here
            self.W -= lr*gradient # do it for sigmoid too
            print((gradient**2).sum())

    def loss_plot(self):
        return self.loss_steps.plot(
            x='step', 
            y='loss',
            xlabel='step',
            ylabel='loss'
        )
    
    def forward(self, X):
        Z = relu(X @ self.W) # todo
        return Z

    def predict(self, X):
        X = add_intercept(X)
        Z = self.forward(X)
        P = softmax(Z, axis=1)
        return np.argmax(P, axis=1)

In [17]:
mlr=MulticlassReLu()

mlr.fit(X_train,Y_train)
y_pred=mlr.predict(X_train)

(22, 3)
1511.374815016564
653.6930151205706
298.2293934569574
153.10027770903656
89.6891137138563
59.68697795085308
44.45465119471146
36.204719601987755
31.435422358564832
28.476557558252498


In [18]:
new_pred = np.zeros((y_pred.shape[0], 3))
for i in range(y_pred.shape[0]):
    idx = y_pred[i]
    new_pred[i][int(idx)] = 1
y_pred = new_pred

In [19]:
from sklearn.metrics import accuracy_score , f1_score
mlr_accuracy = accuracy_score(Y_train,y_pred)
mlr_f1 = f1_score(Y_train,y_pred, average='micro')
print('ACCURACY:',round(mlr_accuracy*100,2),'%')
print('F1 Score:',mlr_f1)

ACCURACY: 64.94 %
F1 Score: 0.6494117647058824


In [20]:
#With above results, sigmoid function gives better accuracy for single neural network (single layer) compare to ReLu activation function